# Sugestão de preço em um apartamento em Manhatam
Caracteristicas: 
{'id': 2595,
 'nome': 'Skylit Midtown Castle',
 'host_id': 2845,
 'host_name': 'Jennifer',
 'bairro_group': 'Manhattan',
 'bairro': 'Midtown',
 'latitude': 40.75362,
 'longitude': -73.98377,
 'room_type': 'Entire home/apt',
 'price': 225,
 'minimo_noites': 1,
 'numero_de_reviews': 45,
 'ultima_review': '2019-05-21',
 'reviews_por_mes': 0.38,
 'calculado_host_listings_count': 2,
 'disponibilidade_365': 355}

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [3]:
df = pd.read_csv('/home/leticia/teste_indicium_precificacao.csv', encoding='utf-8')

In [3]:
#removendo as linhas NaN das colunas nome e host_nome
df = df.dropna(subset=['nome', 'host_name'])

In [4]:
#converte string para datetime
df['ultima_review'] = pd.to_datetime(df['ultima_review'], errors='coerce')

In [5]:
#Preenchendo NaN em ultima_review com uma data padrão, pois são muitos dados faltantes
df['ultima_review'].fillna(df['ultima_review'].min(), inplace=True)

In [6]:
#preenchendo os dados NaN em reviews_por_mes com 0
df['reviews_por_mes'].fillna(0, inplace=True)

In [7]:
#removendo caracteres especiais
import re
df['nome'] = df['nome'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x))

In [8]:
#manipulação dos emojis - desemojizando 
import emoji
df['nome'] = df['nome'].apply(lambda x: emoji.demojize(x))

In [9]:
#eliminando as linhas que tem preço zerado - eram 11
df = df[df['price'] != 0]

In [10]:
criterios = df[(df['bairro_group'] == 'Manhattan') & (df['bairro'] == 'Midtown')]

In [11]:
X = criterios.drop(['price', 'ultima_review'], axis=1) 
y = criterios['price']

In [12]:
X = pd.get_dummies(X)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
#testando regressao linear
from sklearn.linear_model import LinearRegression

In [16]:
regressor_linear = LinearRegression()

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_treino)
X_test_scaled = scaler.transform(X_teste)

In [18]:
# Criando e treinando o modelo de regressão linear
modelo = LinearRegression()
modelo.fit(X_train_scaled, y_treino)

LinearRegression()

In [19]:
# Fazendo previsões no conjunto de teste
y_pred = modelo.predict(X_test_scaled)

In [20]:
# Avaliando o desempenho do modelo
erro_medio_quadratico = mean_squared_error(y_teste, y_pred)
print(f"Erro Médio Quadrático: {erro_medio_quadratico}")

Erro Médio Quadrático: 3.143161616067462e+26


In [24]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [22]:
# Criando um modelo XGBoost
modelo_xgboost = XGBRegressor()

In [23]:
# Definindo a grade de hiperparâmetros para pesquisa
parametros_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

In [25]:
# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=modelo_xgboost, param_grid=parametros_grid, scoring='neg_mean_squared_error', cv=5, verbose=2)


In [27]:
# Executando a pesquisa de grade no conjunto de treinamento
grid_search.fit(X_train_scaled, y_treino)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.3s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.3s
[CV] END colsamp

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 0.9, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5], 'n_estimators': [50, 100, 200],
                         'subsample': [0.8, 0.9, 1.0]},
             scoring='neg_mean_squared_error', verbose=2)

In [28]:
# Obtendo os melhores hiperparâmetros encontrados
melhores_parametros = grid_search.best_params_

In [29]:
# Imprimindo os melhores hiperparâmetros
print("Melhores Hiperparâmetros:")
print(melhores_parametros)

Melhores Hiperparâmetros:
{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'subsample': 0.8}


In [33]:
# Fazendo previsões no conjunto de teste usando o modelo com os melhores hiperparâmetros
modelo_final = grid_search.best_estimator_
previsao = modelo_final.predict(X_test_scaled)

In [34]:
# Avaliando o desempenho do modelo final
erro_medio_quadratico_final = mean_squared_error(y_teste, previsao)
print(f"Erro Médio Quadrático (XGBoost Final): {erro_medio_quadratico_final}")

Erro Médio Quadrático (XGBoost Final): 110893.47447725522


In [37]:
print(previsao.mean())

268.28397
